Install dependencies

In [97]:
! pip install -r requirements.txt --quiet

Load documents and split into chunks

In [78]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


loader = DirectoryLoader("data/")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=150)
documents = loader.load()
texts = text_splitter.split_documents(documents)
len(texts)

804

Semantic chunking

In [98]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from config import azure_openai_embedding_deployment

embeddings = AzureOpenAIEmbeddings(
    deployment=azure_openai_embedding_deployment,
    chunk_size=1024
)

loader = DirectoryLoader("data/")
documents = loader.load()

In [99]:
# documents[0].page_content
text_splitter = SemanticChunker(embeddings=embeddings,
                                breakpoint_threshold_type='interquartile')

chunks = text_splitter.create_documents([documents[i].page_content for i in range(len(documents))])

In [100]:
len(chunks)

121

Create embeddings and save to folder

In [107]:
import os
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from config import azure_openai_embedding_deployment
from langchain_chroma import Chroma
from config import azure_openai_key, azure_openai_endpoint, azure_openai_api_version
import os

os.environ["AZURE_OPENAI_API_KEY"] = azure_openai_key
os.environ["AZURE_OPENAI_ENDPOINT"] = azure_openai_endpoint
os.environ["AZURE_OPENAI_API_VERSION"] = azure_openai_api_version

persist_directory = "chroma"

embeddings = AzureOpenAIEmbeddings(
    deployment=azure_openai_embedding_deployment,
    chunk_size=1024
)

if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)

# vectorstore = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory)
vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory=persist_directory)
# db = Chroma()
# db.from_documents

OperationalError: no such table: embeddings_queue_config

Create persist folder for reusing model

In [74]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from config import azure_openai_gpt_deployment, azure_openai_api_version
from langchain.embeddings import AzureOpenAIEmbeddings


persist_directory = "chromaDB"

embeddings = AzureOpenAIEmbeddings(
    deployment=azure_openai_embedding_deployment,
    chunk_size=1024
)

vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

gpt = AzureChatOpenAI(
    deployment_name=azure_openai_gpt_deployment,
    api_version=azure_openai_api_version
)

qa_chain = RetrievalQA.from_chain_type(
    llm=gpt,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

prompt = """
System:
```
You are a search assistant.
You are an expert in "Consistent evolvement".
You will be asked a question.
If the question comes with answer choicec, you should choose the correct one and explain why.
Provide the full text from the retrieved documents that supports your answer.
All information will be provided in Ukrainian language.
Answer in Ukrainian language.
You should not use any external resources or make up information.
If you cannot find the answer, you should say 
"Відповіді не знайдено. Використаю зовнішні ресурси" 
and give an answer that you find online.
You should think step by step and provide your thought process after the answer.
When you wrote 10 words in 1 line, you should start a new line.
Every line should contain no more than 10 words.
Example:
```
---Відповідь---
...
---Пояснення---
...
---Думки---
...
```

```
"""


# Example query
user = """
User:
```
Що таке сталий розвиток?
```
"""
query = prompt + user + "Answer:"
answer = qa_chain({"query": query})
print(answer.get("result"))
print()
print(answer.get("source_documents"))

---Відповідь---
Сталий розвиток можна характеризувати як модель розвитку або концепцію, яка передбачає підтримку довкілля, не порушуючи його.

---Пояснення---
В контексті української мови сталий розвиток часто перекладають як збалансований, стійкий або підтримуваний розвиток. Основна ідея полягає в підтримуваності, а не сталість.

---Думки---
Поняття сталого розвитку включає в себе екологічну підтримку, економічну ефективність та соціальну справедливість.

[Document(metadata={'source': 'data/lect_1/yt_lect_1.txt'}, page_content="довкілля який підтримує людину так не порушуючи його і відповідно так склалося що в українській мові існує багато різних А варіацій як перекласти правда не жодного коректного тобто то у нас ви можете зустріти переклади збалансований розвиток а сталий розвиток стійкий розвиток останні Ну можливо 5 тире 100 років в державних документах так залишився термін сталий розвиток тому ми будемо притримуватись саме такої термінології але ви повинні розуміти що там не мова

Load chatGPT model

Create prompt

In [20]:

query = """
    Що таке сталий розвиток?
"""
result = qa_chain.run(query)

print(result)